<center>Copyright 2020 Parkland Health & Hospital System </center>

This program entitled “Parkland Trauma Index of Mortality” is free software and is distributed under the terms of the GNU Lesser General Public License (LGPL). You can redistribute it and/or modify it under the terms of the GNU LGPL as published by the Free Software Foundation, either version 3 of the License or any later version. This program is distributed WITHOUT ANY WARRANTY; without even THE IMPLIED WARRANTY OF MERCHANTABILITY or FITTNESS FOR A PARTICULAR PURPOSE. See the GNU LGPL for more details. You should have received a copy of the GNU LGPL along with this program; if not, see https://www.gnu.org/licenses.

In [ ]:
# Load Dependencies
from time import time
import pandas as pd
import copy, sys
import numpy as np
import sqlite3 as lite
import collections
import scikitplot
import sklearn # 0.21.3  Balanced bagging classifier from imblearn needs this version of sklearn for sklearn.externals.joblib
import six
sys.modules['sklearn.externals.six'] = six
from imblearn.ensemble import BalancedBaggingClassifier
from  scikitplot.metrics import plot_confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,  GroupKFold, StratifiedKFold
from sklearn.pipeline import make_pipeline
from scikitplot import classifier_factory
from sklearn.metrics import roc_curve,roc_auc_score, precision_score, recall_score, f1_score,classification_report,confusion_matrix, accuracy_score, precision_recall_curve
import joblib
import lime
import lime.lime_tabular
pd.options.mode.chained_assignment = None
pd.options.display.max_rows = 100
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [ ]:
# Global variables
## Set number of hours to use when resampling clinical data
TIME_BINS = '12H'

## Set maximum number of time periods to include per encounter
MAX_PERIODS = 6
TRUNCATE_LONG_STAYS = 1
IMPUTE = 0

## Set minimum number of hours admitted before prediction
MIN_LOS = 12

## Set number of jobs to use in parallel (-1 = max)
N_JOBS = -1

## Set base path for raw data
DATA_PATH = 'path to data file'

## Set default SQLite database file; default will be in same path as raw data files
SQLITE_DB = DATA_PATH + "trauma_mortality.db"

In [ ]:
## Write selected dataframe to default SQLite database, replacing if found
def DataFrameToSQL(df, df_name):
    connection = lite.connect(SQLITE_DB)
    with connection:
        df.to_sql(df_name, connection, schema = None, if_exists='replace', index=False)
        print('%s backed up to default SQLite database.' % df_name)

In [ ]:
## Retrieve SQL table from default SQLite database and store as a dataframe
def SQLToDataFrame(table_name, date_columns=[]):
    connection = lite.connect(SQLITE_DB)
    sql = 'select * from ' + table_name
    df = pd.read_sql(sql, connection, parse_dates=date_columns)
    return(df)

In [ ]:
## Retrieves raw data for any patient who either has a ta (OR) st with all of the features defined or built in data preparation file.
raw_data = SQLToDataFrame(table_name='merge_all', date_columns=['date_of_arrival', 'current_datetime', 'current_datetime_min', 'birthdate', 'deathdate', 'hosp_admsn_time', 'hosp_dischrg_time','adm_date'])

In [ ]:
## Creating a copy of raw data before making any changes since raw data can take some time to be imported into python.
# Main Cohort: data
data = raw_data.copy()
data['time_elapsed'] = data.time_sequence * np.timedelta64(12, 'h')

In [ ]:
#Read in data of a patient with any of the ta levels mentioned
ta_level123 =  pd.read_csv(DATA_PATH + 'Trauma_Cohort_L123Only.csv')

## PREPROCESSING

In [ ]:
#Exclude any patient under 18
data = data[data.age>= 18.0]  

In [ ]:
# Retrieve level 1 ta and merge it with labs, vitals, etc., from data. # TRAUMA ACTIVATION LEVEL 1 only
ta_level1= ta_level123[ta_level123.TraumaTypeMerged.isin(['Level 1'])]
ta1_data = pd.merge(ta_level1,data, how = 'left', left_on = 'EncounterEpicCsn', right_on = 'pat_enc_csn_id')


In [ ]:
## COPY the above retrieved dataframes to 'data' below. Just to be safe about not messing the above cohort and any code below.
#data = ta_and_st1.copy()
data = ta1_data.copy()
data.pat_enc_csn_id.nunique()

In [ ]:
## Drop rows with missing dependent variable
data.dropna(axis=0, how='any', thresh=None, subset=['death_flag_next_period'], inplace=True)
data.shape

In [ ]:
## two periods has to include next period
data['death_flag_two_periods'] = (data['death_flag_two_periods'] + data['death_flag_next_period'] >= 1.0).astype(int)
data['death_flag_three_periods'] = (data['death_flag_three_periods'] + data['death_flag_two_periods'] >= 1.0).astype(int)
data['death_flag_four_periods'] = (data['death_flag_four_periods'] + data['death_flag_three_periods'] >= 1.0).astype(int)

In [ ]:
## Truncate long tail of hospital stays using MAX_PERIODS global variable
if TRUNCATE_LONG_STAYS == 1:
    data = data.set_index(['primarymrn', 'pat_enc_csn_id', 'time_sequence']).drop(list(range(MAX_PERIODS+1, 1000)), level='time_sequence').reset_index()
    print(data.shape)

In [ ]:
## Drop last observation
data = data.loc[data.death_flag_this_period != 1]
print(data.shape)

## Remove stays shorter than 12 hours; should already be fixed by the above
data = data[data.groupby(['primarymrn','pat_enc_csn_id'])['time_sequence'].transform(max) >= int(MIN_LOS / 12)]
print(data.shape)

## Include only time_sequence > MIN_LOS
data = data[data.time_sequence >= int(MIN_LOS/12)]



In [ ]:
# Remove bad cases where only 1 time sequence is available
#data = data[~((data.time_sequence == data.groupby(['pat_enc_csn_id']).time_sequence.transform(max)) & (data.time_sequence== 0))]

In [ ]:
y_column =  'death_flag_four_periods' # Number of 12 hour intervals to be predicted (4 = 48 hr prediction time)

X_columns = ['pat_enc_csn_id', 
             'time_sequence', 'age', 
             'temperature_max', 'pkmod_r_cpn_glasgow_coma_scale_score',
             'hemoglobin_min', 'potassium_min', 'potassium_max',
             'pulse_max', 'wbc_max', 'lactate_max', 'inr_max', 'creatinine_max',
             'ast_max',  'bilirubin_total_max', 'systolic_max',
             'pulse_min', 'pulse_oximetry_min',
             'platelets_min', 'base_exc_art_min', 'albumin_min', 'systolic_min',
             'pulse_avg', 'pulse_oximetry_avg', 
            'arrival_year'
             ]
features_cat = ['time_sequence'] 
features_cont_float = [
             'temperature_max', 
             'hemoglobin_min', 'potassium_min', 'potassium_max',
             'pulse_max', 'wbc_max', 'lactate_max', 'inr_max', 'creatinine_max',
             'ast_max',  'bilirubin_total_max', 'systolic_max',
             'pulse_min', 'pulse_oximetry_min',
             'platelets_min', 'base_exc_art_min', 'albumin_min', 'systolic_min',
             'pulse_avg', 'pulse_oximetry_avg']

features_cont_int = ['age','pkmod_r_cpn_glasgow_coma_scale_score']



In [ ]:
if y_column == 'death_flag_four_periods':
    data[data['death_flag_next_period'] == 1]['death_flag_four_periods'] = 1
    data[data['death_flag_two_periods'] == 1]['death_flag_four_periods'] = 1
    data[data['death_flag_three_periods'] == 1]['death_flag_four_periods'] = 1

In [ ]:
data_ = data.copy().reset_index()

features = X_columns
X = data_[X_columns]

In [ ]:
if IMPUTE == 1:
    imp = SimpleImputer(missing_values = 'NaN' , strategy = 'median', axis=0, copy=True)
    X = imp.fit_transform(X)    
    
else:
    X = X.fillna(method = 'pad', limit = 2)
    X = X.fillna(-9999)

In [ ]:
Y = data_[[y_column, 'arrival_year']]

In [ ]:
X.pat_enc_csn_id.nunique()

## Modeling Setup

In [ ]:
N_JOBS = 7
K_FOLDS = 3
TEST_SIZE = .25
SEED = 13

BALANCE_RATIO = 1

In [ ]:
X_train_resampled = X.loc[X.arrival_year.isin([2009, 2010, 2011, 2012, 2013,2014])]
X_test_holdout = X.loc[X.arrival_year.isin([2015, 2016])]
Y_train_resampled = Y.loc[Y.arrival_year.isin([2009, 2010, 2011, 2012, 2013,2014])][y_column]
Y_test_holdout = Y.loc[Y.arrival_year.isin([2015, 2016])][y_column]


In [ ]:
X_test_holdout.pat_enc_csn_id.nunique()

In [ ]:
X_train_sd = X_train_resampled.drop(['pat_enc_csn_id', 'arrival_year'], axis = 1).values

In [ ]:
f_names = X_train_resampled.drop(['pat_enc_csn_id', 'arrival_year'], axis = 1).columns

In [ ]:
## TA - Level 1 Only as cohort
# BalancedBaggingClassifier(base_estimator=None, bootstrap=True,
#              bootstrap_features=False, max_features=1.0, max_samples=0.6,
#              n_estimators=200, n_jobs=7, oob_score=True, random_state=13,
#              ratio=0.994436655762512, replacement=True, verbose=False,
#              warm_start=False) 
#ratio=0.9920599001135434

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16,9]
plt.rcParams.update({'font.size': 22})
plt.rcParams.update({'axes.labelsize': 'medium'})

try:
    features.remove('pat_enc_csn_id')
except:
    pass


N_ESTIMATORS = 175 #700
MAX_DEPTH = 8 #4
MIN_SAMPLES_SPLIT = 10
MIN_SAMPLES_LEAF = 4 #4
MAX_FEATURES = 1.0 
CRITERION = 'entropy' 
WARM_START = False
CLASS_WEIGHT = {1: RATIO}  

model_list = [
BalancedBaggingClassifier(base_estimator=None, bootstrap=True,
             bootstrap_features=False, max_features=1.0, max_samples=0.6,
             n_estimators=200, n_jobs=7, oob_score=True, random_state=13,
             ratio=0.994436655762512, replacement=True, verbose=False,
             warm_start=False)
]       

for model in model_list:
    rfr = model
    rfr.fit(X_train_resampled.drop(['pat_enc_csn_id','arrival_year'],axis = 1), Y_train_resampled)
    
    ##Save to pickle file
    #joblib.dump(rfr,'trauma.pkl')
    
    print(y_column)
    print ("Model: %s " % str(rfr).split('(')[0])
    print ("Accuracy Score (Test): %3.3f" % np.round(rfr.score(X_test_holdout.drop(['pat_enc_csn_id','arrival_year'],axis = 1), Y_test_holdout), 3))

    try:
        importances = pd.DataFrame({'feature':X_columns,
            'importance':np.round(rfr.feature_importances_,3)})
        importances = importances.sort_values('importance',ascending=False).set_index('feature')
        print (importances[:5])
    except:
        pass

    predicted = rfr.predict(X_test_holdout.drop(['pat_enc_csn_id','arrival_year'],axis = 1))
    print(sklearn.metrics.classification_report(Y_test_holdout, predicted))

    rfr2=copy.deepcopy(rfr)
    Y_probas = rfr2.predict_proba(X_test_holdout.drop(['pat_enc_csn_id','arrival_year'],axis = 1).values) #as_matrix())#[:,1]

    print(sklearn.metrics.roc_auc_score(Y_test_holdout, Y_probas[:,1]))
    
    scikitplot.metrics.plot_roc(Y_test_holdout, Y_probas, plot_micro=False, plot_macro = False, classes_to_plot = [1])
    plt.legend(['AUC: 0.94'], loc = 'lower right' )
    plt.savefig('ROC_Curve_BBC.jpg', transparent = True, bbox_inches = 'tight')
    
    scikitplot.metrics.plot_precision_recall(Y_test_holdout, Y_probas, plot_micro = False,classes_to_plot=[1])
    plt.legend(['Area: 0.377'], loc = 'lower right')
    plt.savefig('PR_Curve_BBC.jpg', transparent = True, bbox_inches = 'tight')
    
    scikitplot.metrics.plot_cumulative_gain(Y_test_holdout, Y_probas)
    plt.savefig('GainChart_BBC.jpg', transparent = True, bbox_inches = 'tight')
    
    scikitplot.metrics.plot_lift_curve(Y_test_holdout, Y_probas)
    plt.savefig('Lift_Curve_BBC.jpg', transparent = True, bbox_inches = 'tight')
    
    scikitplot.metrics.plot_confusion_matrix(Y_test_holdout, predicted, normalize = False)
    plt.savefig('ConfusionMatrix_BBC.jpg', transparent = True, bbox_inches = 'tight')
    xt = X_train_resampled.drop(['pat_enc_csn_id','arrival_year'],axis = 1).copy()
    
    
    #scikitplot.estimators.plot_feature_importances(rfr2,X_columns)
    plt.show()
    Y_test_preds = pd.DataFrame(Y_test_holdout.copy())
    Y_test_preds['Predictions'] = predicted


In [ ]:
X_test_sd = X_test_holdout.drop(['pat_enc_csn_id', 'arrival_year'], axis =1 )

In [ ]:
feature_dict = {}
for i, j in zip(np.round(np.mean([est.steps[1][1].feature_importances_ for est in rfr.estimators_], axis=0), 3), features):
    feature_dict[j] = i

feat_imp = pd.DataFrame.from_dict(feature_dict, orient='index').sort_values(0, ascending=False)#.to_clipboard()

In [ ]:
feat_imp = feat_imp.reset_index()
feat_imp.columns = ['feature','importance']
feat_imp = feat_imp.replace(['temperature_max','age','time_sequence','pkmod_r_cpn_glasgow_coma_scale_score','hemoglobin_min','potassium_min','potassium_max','pulse_max','wbc_max','lactate_max','inr_max','creatinine_max','ast_max', 'bilirubin_total_max','systolic_max','pulse_min','pulse_oximetry_min','platelets_min','base_exc_art_min','albumin_min','systolic_min','pulse_avg','pulse_oximetry_avg'],
                 ['Maximum Temperature','Age','Time Since Arrival','Glasgow Coma Scale','Minimum Hemoglobin','Minimum Potassium','Maximum Potassium','Maximum Pulse','Maximum WBC','Maximum Lactate','Maximum INR','Maximum Creatinine', 'Maximum AST', 'Maximum Total Bilirubin','Maximum Systolic','Minimum Pulse','Minimum Pulse Oximetry', 'Minimum Platelets','Minimum Base Deficit', 'Minimum Albumin','Minimum Systolic','Average Pulse','Average Pulse Oximetry'])

In [ ]:
# PLOTTING FEATURE IMPORTANCE
fig, ax = plt.subplots()
fig.set_size_inches(12, 9)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Relative Importance')
plt.grid(axis = 'x', linewidth = 0.2)
plt.barh(feat_imp.sort_values('importance')['feature'],feat_imp.sort_values('importance')['importance'], color = 'black')
plt.savefig('Feature_importance_bbc_manuscript.jpg', transparent = True, bbox_inches = 'tight')